In [1]:
!pip install -U langchain_ollama sentence-transformers faiss-cpu pypdf2

In [1]:
from sentence_transformers import SentenceTransformer
from langchain_ollama import ChatOllama
from langchain.text_splitter import CharacterTextSplitter
from PyPDF2 import PdfReader
import numpy as np
import faiss

pdf_path = "SaitejaReddyKonda_Resume.pdf"
reader = PdfReader(pdf_path)
text = "\n".join([page.extract_text() for page in reader.pages])

splitter = CharacterTextSplitter(chunk_size=512, chunk_overlap=50)
texts = splitter.split_text(text)

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = [embedding_model.encode(chunk) for chunk in texts]

dimension = len(embeddings[0])
faiss_index = faiss.IndexFlatL2(dimension)
faiss_index.add(np.array(embeddings).astype('float32'))

def search(query, top_k=3):
    query_vec = embedding_model.encode([query]).astype('float32')
    distances, indices = faiss_index.search(query_vec, top_k)
    return [texts[i] for i in indices[0]]

llm = ChatOllama(model="qwen3:8b")

def ask_resume(question):
    context = "\n".join(search(question, top_k=3))
    prompt = f"Resume details:\n{context}\n\nQ: {question}\nA:"
    response = llm.invoke(prompt)
    return response

answer = ask_resume("List all the projects mentioned in this resume.")
print(answer)

def ask_resume(question):
    context: "\n".join(search(question, top_k=3))
    prompt = f"Resume detail:\n{context}

SyntaxError: unterminated f-string literal (detected at line 40) (2180571708.py, line 40)

In [4]:
with open("rag_output.md", "w", encoding="utf-8") as f:
    f.write("# RAG Output\n\n")
    f.write("**Question:** List all the projects mentioned in this resume.\n\n")
    f.write(f"**Answer:** {answer.content}\n")